In [1]:
# Load packages
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import os
import scipy as scp
import scipy.stats as scps
from datetime import datetime

# Load my own functions
import dnnregressor_train_eval_keras as dnnk
import make_data_wfpt as mdw
from kde_training_utilities import kde_load_data

In [2]:
# Make dnnk class (cpm for choice probability model)
cpm = dnnk.dnn_trainer()

# Load data
data_folder = os.getcwd() + '/data_storage/kde/kde_training_dat/ddm_final_train_test'

# rt_choice
cpm.data['train_features'], cpm.data['train_labels'], cpm.data['test_features'], cpm.data['test_labels'] = kde_load_data(folder = data_folder)


In [3]:
# Make all parameters we can specify explicit
# Model parameters
cpm.model_params

{'input_shape': 3,
 'output_shape': 1,
 'output_activation': 'sigmoid',
 'hidden_layers': [20, 20, 20, 20],
 'hidden_activations': ['relu', 'relu', 'relu', 'relu'],
 'l1_activation': [0.0, 0.0, 0.0, 0.0],
 'l2_activation': [0.0, 0.0, 0.0, 0.0],
 'l1_kernel': [0.0, 0.0, 0.0, 0.0],
 'l2_kernel': [0.0, 0.0, 0.0, 0.0],
 'optimizer': 'Nadam',
 'loss': 'mse',
 'metrics': ['mse']}

In [4]:
# Parameters governing training
cpm.train_params

{'callback_funs': ['ReduceLROnPlateau', 'EarlyStopping', 'ModelCheckpoint'],
 'plateau_patience': 10,
 'min_delta': 0.0001,
 'early_stopping_patience': 15,
 'callback_monitor': 'loss',
 'min_learning_rate': 1e-07,
 'red_coef_learning_rate': 0.1,
 'ckpt_period': 10,
 'ckpt_save_best_only': True,
 'ckpt_save_weights_only': True,
 'max_train_epochs': 2000,
 'batch_size': 10000,
 'warm_start': False,
 'checkpoint': 'ckpt',
 'model_cnt': 0}

In [5]:
# Parameters concerning data storage
cpm.data_params

{'data_type': 'choice_probabilities',
 'model_directory': '/home/afengler/git_repos/nn_likelihoods/keras_models',
 'checkpoint': 'ckpt',
 'model_name': 'dnnregressor',
 'data_type_signature': '_choice_probabilities_analytic_',
 'timestamp': '04_23_19_15_27_54',
 'training_data_size': 2500000}

In [7]:
# SPECIFYING META PARAMETERS THAT STAY CONSTANT DURING HYPERPARAMETER OPTIMIZATION

# Model params
cpm.model_params['output_activation'] = 'linear'
cpm.model_params['input_shape'] = cpm.data['train_features'].shape[1]

# Training params
# Meta
cpm.train_params['early_stopping_patience'] = 10
cpm.train_params['plateau_patience'] = 5
cpm.train_params['min_delta'] = 0.00001
cpm.train_params['ckpt_period'] = 1
cpm.train_params['model_cnt'] = 0
cpm.train_params['max_train_epochs'] = 120

# Data params
cpm.data_params['data_type'] = 'wfpt'
cpm.data_params['data_type_signature'] = '_kde_ddm_small_fan_out_'
cpm.data_params['training_data_size'] = cpm.data['train_features'].shape[0]

# Update timestamp
cpm.data_params['timestamp'] = datetime.now().strftime('%m_%d_%y_%H_%M_%S')

In [ ]:
# Make model
# cpm.keras_model_generate(save_model = True)

In [8]:
# Hyperparameter training loop:

# Runs: 
num_runs = 3
cnt = 0
max_layers = 7
layer_sizes = [20, 40, 60, 80]
batch_sizes = [10000, 100000, 200000]
regularization_sizes = [0.0, 0.0]

# Update model directory to make sure we collect all our models from this hyperparameter optimization run in the same place
cpm.data_params['model_directory'] =  '/home/afengler/git_repos/nn_likelihoods/keras_models/'
cpm.data_params['model_name'] = 'ddm_kde_hyp_opt_test'
cpm.train_params['model_cnt'] = 0

histories = []

while cnt < num_runs:
    
    # Sample # layers 
    num_layers = 4
    #num_layers = np.random.choice(np.arange(max_layers, max_layers + 1, 1))
    
    # Layer sizes
    # Generate new architecture
    layers = []
    activations = []
    regularizers_l1 = []
    regularizers_l2 = []
    
    for i in range(0, num_layers, 1):
        #layers.append(np.random.choice(layer_sizes))
        layers.append(layer_sizes[i])
        activations.append('relu')
        regularizers_l1.append(0.0)
        regularizers_l2.append(0.0)
        
    # Batch size
    #batch_size = np.random.choice(batch_sizes)
    batch_size = batch_sizes[cnt]
    
    # Update relevant hyperparameters in training class 
    cpm.train_params['batch_size'] = batch_size
    print('batch_size: ', batch_size)
    cpm.model_params['hidden_layers'] = layers
    print('layers: ', layers)
    cpm.model_params['hidden_activations'] = activations
    print('hidden_activations:', activations)
    cpm.model_params['l1_activation'] = regularizers_l1
    print('l1_activatons: ', regularizers_l1)
    cpm.model_params['l2_activation'] = regularizers_l2
    print('l2_activations:', regularizers_l2)
    cpm.model_params['l1_kernel'] = regularizers_l1
    print('l1_kernel: ', regularizers_l1)
    cpm.model_params['l2_kernel'] = regularizers_l2
    print('l2_kernel: ', regularizers_l2)
    
    # Make model
    cpm.keras_model_generate(save_model = True)
    
    # Train model
    cpm.run_training(save_history = True, 
                     warm_start = False) # Note that this increments model count automatically !
    
    # Print model count
    print(cnt)
    cnt += 1

batch_size:  10000
layers:  [20, 40, 60, 80]
hidden_activations: ['relu', 'relu', 'relu', 'relu']
l1_activatons:  [0.0, 0.0, 0.0, 0.0]
l2_activations: [0.0, 0.0, 0.0, 0.0]
l1_kernel:  [0.0, 0.0, 0.0, 0.0]
l2_kernel:  [0.0, 0.0, 0.0, 0.0]
Train on 108125520 samples, validate on 27033480 samples
Epoch 1/120
108125520/108125520 [==============================] - 133s 1us/step - loss: 0.3899 - mean_squared_error: 0.3899 - val_loss: 0.1043 - val_mean_squared_error: 0.1043

Epoch 00001: val_loss improved from inf to 0.10425, saving model to /home/afengler/git_repos/nn_likelihoods/keras_models//ddm_kde_hyp_opt_test_kde_ddm_small_fan_out_04_23_19_15_28_09/ckpt_0_01
Epoch 2/120
108125520/108125520 [==============================] - 133s 1us/step - loss: 0.0851 - mean_squared_error: 0.0851 - val_loss: 0.1204 - val_mean_squared_error: 0.1204

Epoch 00002: val_loss did not improve from 0.10425
Epoch 3/120
108125520/108125520 [==============================] - 134s 1us/step - loss: 0.0761 - mean_sq

108125520/108125520 [==============================] - 133s 1us/step - loss: 0.0089 - mean_squared_error: 0.0089 - val_loss: 0.0071 - val_mean_squared_error: 0.0071

Epoch 00028: val_loss did not improve from 0.00667
Epoch 29/120
108125520/108125520 [==============================] - 133s 1us/step - loss: 0.0087 - mean_squared_error: 0.0087 - val_loss: 0.0107 - val_mean_squared_error: 0.0107

Epoch 00029: val_loss did not improve from 0.00667
Epoch 30/120
108125520/108125520 [==============================] - 133s 1us/step - loss: 0.0085 - mean_squared_error: 0.0085 - val_loss: 0.0070 - val_mean_squared_error: 0.0070

Epoch 00030: val_loss did not improve from 0.00667
Epoch 31/120
108125520/108125520 [==============================] - 133s 1us/step - loss: 0.0086 - mean_squared_error: 0.0086 - val_loss: 0.0080 - val_mean_squared_error: 0.0080

Epoch 00031: val_loss did not improve from 0.00667
Epoch 32/120
108125520/108125520 [==============================] - 133s 1us/step - loss: 0.0

108125520/108125520 [==============================] - 133s 1us/step - loss: 0.0067 - mean_squared_error: 0.0067 - val_loss: 0.0059 - val_mean_squared_error: 0.0059

Epoch 00060: val_loss did not improve from 0.00534
Epoch 61/120
108125520/108125520 [==============================] - 134s 1us/step - loss: 0.0067 - mean_squared_error: 0.0067 - val_loss: 0.0052 - val_mean_squared_error: 0.0052

Epoch 00061: val_loss improved from 0.00534 to 0.00518, saving model to /home/afengler/git_repos/nn_likelihoods/keras_models//ddm_kde_hyp_opt_test_kde_ddm_small_fan_out_04_23_19_15_28_09/ckpt_0_61
Epoch 62/120
108125520/108125520 [==============================] - 134s 1us/step - loss: 0.0067 - mean_squared_error: 0.0067 - val_loss: 0.0059 - val_mean_squared_error: 0.0059

Epoch 00062: val_loss did not improve from 0.00518
Epoch 63/120
108125520/108125520 [==============================] - 134s 1us/step - loss: 0.0071 - mean_squared_error: 0.0071 - val_loss: 0.0060 - val_mean_squared_error: 0.0060

108125520/108125520 [==============================] - 133s 1us/step - loss: 0.0047 - mean_squared_error: 0.0047 - val_loss: 0.0047 - val_mean_squared_error: 0.0047

Epoch 00087: val_loss did not improve from 0.00472
Epoch 00087: early stopping
0
batch_size:  100000
layers:  [20, 40, 60, 80]
hidden_activations: ['relu', 'relu', 'relu', 'relu']
l1_activatons:  [0.0, 0.0, 0.0, 0.0]
l2_activations: [0.0, 0.0, 0.0, 0.0]
l1_kernel:  [0.0, 0.0, 0.0, 0.0]
l2_kernel:  [0.0, 0.0, 0.0, 0.0]
Train on 108125520 samples, validate on 27033480 samples
Epoch 1/120
108125520/108125520 [==============================] - 134s 1us/step - loss: 1.2471 - mean_squared_error: 1.2471 - val_loss: 0.6247 - val_mean_squared_error: 0.6247

Epoch 00001: val_loss improved from inf to 0.62467, saving model to /home/afengler/git_repos/nn_likelihoods/keras_models//ddm_kde_hyp_opt_test_kde_ddm_small_fan_out_04_23_19_15_28_09/ckpt_1_01
Epoch 2/120
108125520/108125520 [==============================] - 135s 1us/step - los

108125520/108125520 [==============================] - 132s 1us/step - loss: 0.0176 - mean_squared_error: 0.0176 - val_loss: 0.0074 - val_mean_squared_error: 0.0074

Epoch 00027: val_loss improved from 0.00785 to 0.00736, saving model to /home/afengler/git_repos/nn_likelihoods/keras_models//ddm_kde_hyp_opt_test_kde_ddm_small_fan_out_04_23_19_15_28_09/ckpt_1_27
Epoch 28/120
108125520/108125520 [==============================] - 133s 1us/step - loss: 0.0174 - mean_squared_error: 0.0174 - val_loss: 0.0175 - val_mean_squared_error: 0.0175

Epoch 00028: val_loss did not improve from 0.00736
Epoch 29/120
108125520/108125520 [==============================] - 134s 1us/step - loss: 0.0169 - mean_squared_error: 0.0169 - val_loss: 0.0090 - val_mean_squared_error: 0.0090

Epoch 00029: val_loss did not improve from 0.00736
Epoch 30/120
108125520/108125520 [==============================] - 135s 1us/step - loss: 0.0166 - mean_squared_error: 0.0166 - val_loss: 0.0083 - val_mean_squared_error: 0.0083

...

108125520/108125520 [==============================] - 135s 1us/step - loss: 0.0051 - mean_squared_error: 0.0051 - val_loss: 0.0051 - val_mean_squared_error: 0.0051

Epoch 00083: val_loss did not improve from 0.00514
Epoch 84/120
108125520/108125520 [==============================] - 136s 1us/step - loss: 0.0051 - mean_squared_error: 0.0051 - val_loss: 0.0051 - val_mean_squared_error: 0.0051

Epoch 00084: val_loss improved from 0.00514 to 0.00514, saving model to /home/afengler/git_repos/nn_likelihoods/keras_models//ddm_kde_hyp_opt_test_kde_ddm_small_fan_out_04_23_19_15_28_09/ckpt_1_84
Epoch 85/120
108125520/108125520 [==============================] - 137s 1us/step - loss: 0.0051 - mean_squared_error: 0.0051 - val_loss: 0.0051 - val_mean_squared_error: 0.0051

Epoch 00085: ReduceLROnPlateau reducing learning rate to 1e-07.

Epoch 00085: val_loss improved from 0.00514 to 0.00514, saving model to /home/afengler/git_repos/nn_likelihoods/keras_models//ddm_kde_hyp_opt_test_kde_ddm_small_fa

108125520/108125520 [==============================] - 123s 1us/step - loss: 0.0386 - mean_squared_error: 0.0386 - val_loss: 0.0252 - val_mean_squared_error: 0.0252

Epoch 00022: val_loss did not improve from 0.02026
Epoch 23/120
108125520/108125520 [==============================] - 121s 1us/step - loss: 0.0366 - mean_squared_error: 0.0366 - val_loss: 0.0392 - val_mean_squared_error: 0.0392

Epoch 00023: val_loss did not improve from 0.02026
Epoch 24/120
108125520/108125520 [==============================] - 121s 1us/step - loss: 0.0331 - mean_squared_error: 0.0331 - val_loss: 0.0192 - val_mean_squared_error: 0.0192

Epoch 00024: val_loss improved from 0.02026 to 0.01919, saving model to /home/afengler/git_repos/nn_likelihoods/keras_models//ddm_kde_hyp_opt_test_kde_ddm_small_fan_out_04_23_19_15_28_09/ckpt_2_24
Epoch 25/120
108125520/108125520 [==============================] - 121s 1us/step - loss: 0.0324 - mean_squared_error: 0.0324 - val_loss: 0.0176 - val_mean_squared_error: 0.0176

108125520/108125520 [==============================] - 123s 1us/step - loss: 0.0091 - mean_squared_error: 0.0091 - val_loss: 0.0091 - val_mean_squared_error: 0.0091

Epoch 00053: val_loss improved from 0.00922 to 0.00910, saving model to /home/afengler/git_repos/nn_likelihoods/keras_models//ddm_kde_hyp_opt_test_kde_ddm_small_fan_out_04_23_19_15_28_09/ckpt_2_53
Epoch 54/120
108125520/108125520 [==============================] - 122s 1us/step - loss: 0.0090 - mean_squared_error: 0.0090 - val_loss: 0.0090 - val_mean_squared_error: 0.0090

Epoch 00054: val_loss improved from 0.00910 to 0.00897, saving model to /home/afengler/git_repos/nn_likelihoods/keras_models//ddm_kde_hyp_opt_test_kde_ddm_small_fan_out_04_23_19_15_28_09/ckpt_2_54
Epoch 55/120
108125520/108125520 [==============================] - 122s 1us/step - loss: 0.0089 - mean_squared_error: 0.0089 - val_loss: 0.0088 - val_mean_squared_error: 0.0088

Epoch 00055: val_loss improved from 0.00897 to 0.00884, saving model to /home/afen

108125520/108125520 [==============================] - 122s 1us/step - loss: 0.0067 - mean_squared_error: 0.0067 - val_loss: 0.0066 - val_mean_squared_error: 0.0066

Epoch 00077: val_loss improved from 0.00664 to 0.00663, saving model to /home/afengler/git_repos/nn_likelihoods/keras_models//ddm_kde_hyp_opt_test_kde_ddm_small_fan_out_04_23_19_15_28_09/ckpt_2_77
Epoch 78/120
108125520/108125520 [==============================] - 123s 1us/step - loss: 0.0066 - mean_squared_error: 0.0066 - val_loss: 0.0067 - val_mean_squared_error: 0.0067

Epoch 00078: val_loss did not improve from 0.00663
Epoch 79/120
108125520/108125520 [==============================] - 120s 1us/step - loss: 0.0066 - mean_squared_error: 0.0066 - val_loss: 0.0065 - val_mean_squared_error: 0.0065

Epoch 00079: val_loss improved from 0.00663 to 0.00654, saving model to /home/afengler/git_repos/nn_likelihoods/keras_models//ddm_kde_hyp_opt_test_kde_ddm_small_fan_out_04_23_19_15_28_09/ckpt_2_79
Epoch 80/120
108125520/10812552


Epoch 00103: val_loss did not improve from 0.00579
Epoch 104/120
108125520/108125520 [==============================] - 122s 1us/step - loss: 0.0059 - mean_squared_error: 0.0059 - val_loss: 0.0059 - val_mean_squared_error: 0.0059

Epoch 00104: val_loss did not improve from 0.00579
Epoch 105/120
108125520/108125520 [==============================] - 123s 1us/step - loss: 0.0059 - mean_squared_error: 0.0059 - val_loss: 0.0060 - val_mean_squared_error: 0.0060

Epoch 00105: val_loss did not improve from 0.00579
Epoch 106/120
108125520/108125520 [==============================] - 121s 1us/step - loss: 0.0058 - mean_squared_error: 0.0058 - val_loss: 0.0057 - val_mean_squared_error: 0.0057

Epoch 00106: val_loss improved from 0.00579 to 0.00571, saving model to /home/afengler/git_repos/nn_likelihoods/keras_models//ddm_kde_hyp_opt_test_kde_ddm_small_fan_out_04_23_19_15_28_09/ckpt_2_106
Epoch 107/120
108125520/108125520 [==============================] - 123s 1us/step - loss: 0.0058 - mean_squ